# <p style="text-align: center;">RNA Knowledge Graph Build Data Preparation</p>
    
***
***

**Authors:** [ECavalleri](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=emanuele.cavalleri@studenti.unimi.it), [TJCallahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com), [MMesiti](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=marco.mesiti@unimi.it)

**GitHub Repositories:** [testRNA-KG](https://github.com/emanuelecavalleri/testRNA-KG), [PheKnowLator](https://github.com/callahantiff/PheKnowLator/)  
<!--- **Release:** **[v2.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)** --->
  
<br>  
  
**Purpose:** This notebook serves as a script to download, process, map, and clean data in order to build edges for the RNA-based Knowledge Graph. For more information on the data sources utilize within this script, please see the [PheKnowLator Data Sources](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources) Wiki page.

<br>

**Assumptions:**   
- Edge data downloads ➞ `./resources/edge_data`  
- Ontologies ➞ `./resources/ontologies`    
- Processed data write location ➞ `./resources/processed_data`  

<br>

**Dependencies:**   
- **Scripts**: This notebook utilizes several helper functions, which are stored in the [`data_utils.py`](https://github.com/callahantiff/PheKnowLator/blob/master/pkt_kg/utils/data_utils.py) and [`kg_utils.py`](https://github.com/callahantiff/PheKnowLator/blob/master/pkt_kg/utils/kg_utils.py) scripts.  
- **Data**: All downloaded and generated data sources are provided through [this](https://drive.google.com/drive/folders/1sev5zczMviX7UVqMhTpkFXG43K3nQa9f) dedicated Google Drive repository. <u>This notebook will download everything that is needed for you</u>.  
_____
***

## Table of Contents
***

### [Download Ontologies](#create-ontologies)


### [Create Identifier Maps ](#create-identifier-maps)   


### [Download and process Edge Datasets](#create-edges)  

____
***

## Set-Up Environment
***

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt
sys.path.append('../')


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip


# TODO: Next run, I'll check if all these libraries are needed here

In [2]:
# import needed libraries
import datetime
import glob
import itertools
import networkx
import numpy
import os
import openpyxl
import pandas
import pickle
import re
import requests
import tarfile
import pandas as pd

from collections import Counter
from functools import reduce
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import OWL, RDF, RDFS
from reactome2py import content
from tqdm import tqdm
from typing import Dict
from typing import Tuple

from pkt_kg.utils import * 
from builds.ontology_cleaning import *

#### Define Global Variables

In [3]:
# directory to store resources
resource_data_location = '../resources/'

# directory to use for processing data
processed_data_location = '../resources/processed_data/'

# directory to write ontology data to
ontology_data_location = '../resources/ontologies/'

# directory to write edges data to
edge_data_location = '../resources/edge_data/'

In [4]:
# Download data function for already processed data
def download(name, path):
    url = 'https://storage.googleapis.com/pheknowlator/current_build/data/processed_data/'+name
    if not os.path.exists(path + name):
        data_downloader(url, path)

In [28]:
# KG construction approach dictionary (for non-Ontology data), provided by PKL ecosystem
download('subclass_construction_map.pkl', '../resources/construction_approach/')

# Load data, print row count, and preview it
nonO_data = pd.read_pickle(r'../resources/construction_approach/'+'subclass_construction_map.pkl')

# For instance, ncbi IDs are mapped to appropriate SO Ontology entries
list(nonO_data.items())[:5]

[('84103', ['SO_0001217']),
 ('84690', ['SO_0001217']),
 ('3579', ['SO_0001217']),
 ('54514', ['SO_0001217']),
 ('7159', ['SO_0001217'])]

In [29]:
# Relations labels are already provided by PKL ecosystem
download('RELATIONS_LABELS.txt', '../resources/relations_data/')

# Load data, print row count, and preview it
ro_data_label = pandas.read_csv('../resources/relations_data/'+'RELATIONS_LABELS.txt', header=0, delimiter='\t')

print('There are {edge_count} RO Relations and Labels'.format(edge_count=len(ro_data_label)))
ro_data_label.head(n=5)

There are 667 RO Relations and Labels


,Label,Relation
0,helper property (not for use in curation),http://purl.obolibrary.org/obo/RO_0002464
1,developmentally replaces,http://purl.obolibrary.org/obo/RO_0002285
2,is approximately equivalent to,http://purl.obolibrary.org/obo/RO_0002603
3,has intracellular endoparasite,http://purl.obolibrary.org/obo/RO_0002641
4,supplies,http://purl.obolibrary.org/obo/RO_0002178


***
***
### DOWNLOAD ONTOLOGIES  <a class="anchor" id="create-ontologies"></a>
***
***

### Relation Ontology ([RO](https://www.ebi.ac.uk/ols/ontologies/ro))
The OBO Relations Ontology (RO) is a collection of OWL relations (ObjectProperties) intended for use across a wide variety of biological ontologies.

In [ ]:
if not os.path.exists(ontology_data_location + 'ro_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/ro.owl',
                             ontology_data_location + 'ro_with_imports.owl'))

***
### Mondo Disease Ontology ([MONDO](https://www.ebi.ac.uk/ols/ontologies/ro))
A semi-automatically constructed ontology that merges in multiple disease resources to yield a coherent merged ontology.

In [ ]:
if not os.path.exists(ontology_data_location + 'mondo_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/mondo.owl',
                             ontology_data_location + 'mondo_with_imports.owl'))

***
### Non-Coding RNA Ontology ([NCRO](https://www.ebi.ac.uk/ols/ontologies/ro))
An ontology for non-coding RNA, both of biological origin, and engineered.

In [ ]:
if not os.path.exists(ontology_data_location + 'ncro_with_imports.owl'):
    command = '{} {} --merge-import-closure -o {}'
    os.system(command.format(owltools_location, 'http://purl.obolibrary.org/obo/ncro.owl',
                             ontology_data_location + 'ncro_with_imports.owl'))

***
***
### CREATE MAPPING DATASETS  <a class="anchor" id="create-identifier-maps"></a>
***
***

### Ensembl Gene-Entrez Gene <a class="anchor" id="ensemblgene-entrezgene"></a>


**Purpose:** To map Ensembl gene identifiers to Entrez gene identifiers

**Output:** `ENSEMBL_GENE_ENTREZ_GENE_MAP.txt`

Already provided by PKL ecosystem.

In [5]:
download('ENSEMBL_GENE_ENTREZ_GENE_MAP.txt', processed_data_location)

ensEntrez = pd.read_csv(processed_data_location + 'ENSEMBL_GENE_ENTREZ_GENE_MAP.txt', sep="\t", header=None)
ensEntrez

,0,1,2,3,4,5
0,ENSG00000171241,79801,protein-coding,protein-coding,protein-coding,protein-coding
1,ENSG00000131149,23199,protein-coding,protein-coding,protein-coding,protein-coding
2,ENSG00000096092,28978,protein-coding,protein-coding,protein-coding,protein-coding
3,ENSG00000222691,106479891,snRNA,pseudogene,not protein-coding,not protein-coding
4,ENSG00000230052,100873180,unprocessed_pseudogene,pseudogene,not protein-coding,not protein-coding
...,...,...,...,...,...,...
42283,ENSG00000175699,256369,protein-coding,protein-coding,protein-coding,protein-coding
42284,ENSG00000251308,359776,processed_pseudogene,pseudogene,not protein-coding,not protein-coding
42285,ENSG00000108479,2584,protein-coding,protein-coding,protein-coding,protein-coding
42286,ENSG00000167371,112476,protein-coding,protein-coding,protein-coding,protein-coding


***
### Disease Ontology (DO) - MONDO mapping <a class="anchor" id="ensemblgene-entrezgene"></a>


**Purpose:** To map DO identifiers to MONDO identifiers

**Output:** `DISEASE_DOID_MONDO_Map.txt`

In [ ]:
mondo_graph = Graph().parse(ontology_data_location + 'mondo_with_imports.owl')

dbxref_res = gets_ontology_class_dbxrefs(mondo_graph)[0]

# Fix DOIDs (substitute : with _) and upper case them
mondo_dict = {str(k).replace(':','_').upper(): {str(i).split('/')[-1].replace(':','_') for i in v}
              for k, v in dbxref_res.items() if 'doid' in str(k)}

with open(processed_data_location + 'DISEASE_DOID_MONDO_Map.txt', 'w') as outfile:
    for k, v in mondo_dict.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

In [6]:
doidMondo = pd.read_csv(processed_data_location + 'DISEASE_DOID_MONDO_Map.txt', sep="\t", header=None)
doidMondo

,0,1
0,DOID_0060503,MONDO_0000778
1,DOID_0111559,MONDO_0032728
2,DOID_3635,MONDO_0018940
3,DOID_0110321,MONDO_0013200
4,DOID_10444,MONDO_0001037
...,...,...
9786,DOID_0111153,MONDO_0016558
9787,DOID_13404,MONDO_0007011
9788,DOID_0111341,MONDO_0007434
9789,DOID_14271,MONDO_0001930


***
### Disease description from DO - DO mapping <a class="anchor" id="ensemblgene-entrezgene"></a>


**Purpose:** To map Disease descriptions from DO to DO identifiers

**Output:** None, this mapping will be used only internally.

Provided by [mir2Disease](http://watson.compbio.iupui.edu:8080/miR2Disease/).

In [7]:
data_downloader('http://watson.compbio.iupui.edu:8080/miR2Disease/download/diseaseList.txt', processed_data_location)
 
descDOmap = pd.read_csv(processed_data_location + 'diseaseList.txt', sep="\t")
descDOmap

,disease name in original paper,disease ontology ID
0,Abdominal Aortic Aneurysm,DOID:7693
1,acute lymphoblastic leukemia (ALL),DOID:9952
2,acute myeloid leukemia (AML),DOID:9119
3,acute myocardial infarction,DOID:9408
4,acute promyelocytic leukemia (APL),DOID:9119
...,...,...
169,uterine leiomyoma (ULM),DOID:13223
170,uveal melanoma,DOID:1909
171,vascular disease,DOID:178
172,vesicular stomatitis,DOID:10881


Then fix it to be appropriate used by PKL ecosystem.

In [8]:
descDOmap.columns = ['desc', 'doid']
descDOmap['desc'] = descDOmap['desc'].str.lower()
descDOmap['doid'] = descDOmap['doid'].str.upper()
descDOmap['doid'] = descDOmap['doid'].str.replace(':', '_')
descDOmap

,desc,doid
0,abdominal aortic aneurysm,DOID_7693
1,acute lymphoblastic leukemia (all),DOID_9952
2,acute myeloid leukemia (aml),DOID_9119
3,acute myocardial infarction,DOID_9408
4,acute promyelocytic leukemia (apl),DOID_9119
...,...,...
169,uterine leiomyoma (ulm),DOID_13223
170,uveal melanoma,DOID_1909
171,vascular disease,DOID_178
172,vesicular stomatitis,DOID_10881


In [9]:
# Remove original file
os.remove(processed_data_location + 'diseaseList.txt')

***
### miRBase ID - Non-Coding RNA Ontology (NCRO) mapping


**Purpose:** To map miRNA identifiers from miRBase to NCRO identifiers

**Output:** `MIRBASE_ID_NCRO_MAP.txt`

In [ ]:
# Helper function to get dbxrefs for all ontology classes' label
def gets_ontology_class_label(graph: Graph) -> Tuple:
    dbx_uris: Dict = dict()
    dbx = [x for x in graph if 'label' in str(x[1]).lower() if isinstance(x[0], URIRef)]
    for x in dbx:
        if str(x[2]).lower() in dbx_uris.keys(): dbx_uris[str(x[2]).lower()].append(str(x[0]))
        else: dbx_uris[str(x[2]).lower()] = [str(x[0])]
    dbx_type = {str(x[2]).lower(): 'DbXref' for x in dbx}

    ex_uris: Dict = dict()
    ex = [x for x in graph if 'exactmatch' in str(x[1]).lower() if isinstance([0], URIRef)]
    for x in ex:
        if str(x[2]).lower() in ex_uris.keys(): ex_uris[str(x[2]).lower()].append(str(x[0]))
        else: ex_uris[str(x[2]).lower()] = [str(x[0])]
    ex_type = {str(x[2]).lower(): 'ExactMatch' for x in ex}

    return {**dbx_uris, **ex_uris}, {**dbx_type, **ex_type}

In [ ]:
# Read data into RDFLib graph object
ncro_graph = Graph().parse(ontology_data_location + 'ncro_with_imports.owl')

dbxref_res = gets_ontology_class_label(ncro_graph)[0]

# Fix string patterns
ncro_dict = {str(k): {str(i).split('/')[-1].replace(':','_') for i in v}
             for k, v in dbxref_res.items() if 'NCRO' in str(v) and 'mir-' in str(k) and 'hsa' in str(k)}
ncro_dict2 = {'hsa-'+str(k): {str(i).split('/')[-1].replace(':','_') for i in v} for k, v in dbxref_res.items()
              if 'NCRO' in str(v) and 'mir-' in str(k) and 'hsa' not in str(k)}

In [ ]:
with open(processed_data_location + 'MIRBASE_ID_NCRO_MAP.txt', 'w') as outfile:
    for k, v in {**ncro_dict, **ncro_dict2}.items():
        outfile.write(str(k) + '\t' + str(v).replace('{','').replace('\'','').replace('}','') + '\n')

In [11]:
mirbaseNcro = pd.read_csv(processed_data_location + 'MIRBASE_ID_NCRO_MAP.txt', sep="\t", header=None)
mirbaseNcro

,0,1
0,hsa-mir-1302-8,NCRO_0002185
1,hsa-mir-3617,NCRO_0003197
2,hsa-mir-4539,NCRO_0001132
3,hsa-mir-210,NCRO_0002758
4,hsa-mir-6895,NCRO_0001579
...,...,...
2014,hsa-mir-9,NCRO_0002701
2015,hsa-mir-1302,NCRO_0002885
2016,hsa-mir-3180,NCRO_0003084
2017,hsa-mir-10,NCRO_0002713


***
***
### DOWNLOAD EDGES  <a class="anchor" id="create-edges"></a>
***
***

### gene-disease from PKL itself

In [14]:
data_downloader('https://storage.googleapis.com/pheknowlator/current_build/data/original_data/curated_gene_disease_associations.tsv',
                edge_data_location)

# Rename file adding relationship's identifier
os.rename(edge_data_location+'curated_gene_disease_associations.tsv',
          edge_data_location+'gene-disease_curated_gene_disease_associations.tsv')

with open(edge_data_location + 'gene-disease_curated_gene_disease_associations.tsv') as f:
    data = f.read()

data = pd.read_csv(edge_data_location + 'gene-disease_curated_gene_disease_associations.tsv', sep="\t")  
data

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.700,0.538,C0019209,Hepatomegaly,phenotype,C23;C06,Finding,0.30,1.000,2017.0,2017.0,1,0,CTD_human
1,1,A1BG,0.700,0.538,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,1.000,2015.0,2015.0,1,0,CTD_human
2,2,A2M,0.529,0.769,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.50,0.769,1998.0,2018.0,3,0,CTD_human
3,2,A2M,0.529,0.769,C0007102,Malignant tumor of colon,disease,C06;C04,Neoplastic Process,0.31,1.000,2004.0,2019.0,1,0,CTD_human
4,2,A2M,0.529,0.769,C0009375,Colonic Neoplasms,group,C06;C04,Neoplastic Process,0.30,1.000,2004.0,2004.0,1,0,CTD_human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84033,109580095,HBB-LCR,0.743,0.115,C0002875,Cooley's anemia,disease,C16;C15,Disease or Syndrome,0.30,NaN,NaN,NaN,0,0,CTD_human
84034,109580095,HBB-LCR,0.743,0.115,C0005283,beta Thalassemia,disease,C16;C15,Disease or Syndrome,0.30,NaN,NaN,NaN,0,0,CTD_human
84035,109580095,HBB-LCR,0.743,0.115,C0019025,Hemoglobin F Disease,disease,C16;C15,Disease or Syndrome,0.30,NaN,NaN,NaN,0,0,CTD_human
84036,109580095,HBB-LCR,0.743,0.115,C0085578,Thalassemia Minor,disease,C16;C15,Disease or Syndrome,0.30,NaN,NaN,NaN,0,0,CTD_human


***
### gene-miRNA from [TarBase](https://dianalab.e-ce.uth.gr/html/diana/web/index.php?r=tarbasev8/index)
DIANA-TarBase v8 is a reference database devoted to the indexing of experimentally supported microRNA (miRNA) targets.

In [17]:
data_downloader('https://dianalab.e-ce.uth.gr/downloads/tarbase_v8_data.tar.gz', edge_data_location)

with tarfile.TarFile(edge_data_location+'tarbase_v8_data.tar', 'r') as tar_ref:
    tar_ref.extractall(edge_data_location)
    
# Remove tar file
os.remove(edge_data_location+'tarbase_v8_data.tar')
    
# Rename file adding relationship's identifier
os.rename(edge_data_location+'TarBase_v8_download.txt',
          edge_data_location+'gene-miRNA_TarBase_v8_download.txt')   

with open(edge_data_location + 'gene-miRNA_TarBase_v8_download.txt') as f:
    data = f.read()

data = pd.read_csv(edge_data_location + 'gene-miRNA_TarBase_v8_download.txt', sep="\t", dtype={"cell_line": "string"})  
data

,geneId,geneName,mirna,species,cell_line,tissue,category,method,positive_negative,direct_indirect,up_down,condition
0,0910001A06Rik(mmu),0910001A06Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,UP,NaN
1,1200004M23Rik(mmu),1200004M23Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,DOWN,NaN
2,1700027J05Rik(mmu),1700027J05Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,UP,NaN
3,1810015A11Rik(mmu),1810015A11Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,DOWN,NaN
4,2310047A01Rik(mmu),2310047A01Rik(mmu),mmu-miR-124-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,DOWN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
927114,uPA(hsa),uPA(hsa),hsa-miR-23b-3p,Homo sapiens,<NA>,NaN,Cancer/Malignant,Western Blot,POSITIVE,INDIRECT,DOWN,NaN
927115,vimentin(hsa),vimentin(hsa),hsa-miR-9-5p,Homo sapiens,<NA>,NaN,Cancer/Malignant,Western Blot,NEGATIVE,INDIRECT,NaN,NaN
927116,Â Â Â Â (PTPRG)(hsa),Â Â Â Â (PTPRG)(hsa),hsa-miR-146a-5p,Homo sapiens,<NA>,NaN,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,UP,NaN
927117,Î±-actin(mmu),Î±-actin(mmu),mmu-miR-24-3p,Mus musculus,<NA>,NaN,Cancer/Malignant,qPCR,POSITIVE,INDIRECT,UP,NaN


In [18]:
# For the time being, we keep only Homo sapiens rows
data = data[data['species'].str.contains("Homo sapiens")]

# Moreover, we keep only 50k (random) rows to reduce input size
data = data.sample(n=50000)

# NCRO Ontology ignores if a transcript is "3p" or "5p", so we store this information as additional column
data['p'] = data[data['mirna'].str.contains("p")]['mirna']
data["p"] = data["p"].str[-2:]
data["mirna"] = data["mirna"].str.replace(r'-3p$', '')
data["mirna"] = data["mirna"].str.replace(r'-5p$', '')

# If you want to learn more about why we have 5p or 3p human miRNAs:
# https://www.mirbase.org/blog/category/nomenclature/

# Again, for NCRO IDs compatability
data['mirna'] = data['mirna'].str.lower()
data

,geneId,geneName,mirna,species,cell_line,tissue,category,method,positive_negative,direct_indirect,up_down,condition,p
296946,ENSG00000138756,BMP2K,hsa-mir-22,Homo sapiens,HUVEC,Umbilical Vein,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,DOWN,NaN,3p
34726,ENSG00000062650,WAPAL,hsa-mir-3657,Homo sapiens,293S,Kidney,Embryonic/Fetal,HITS-CLIP,POSITIVE,DIRECT,DOWN,treatment: emetine,NaN
145176,ENSG00000109670,FBXW7,hsa-mir-4782,Homo sapiens,HEK293,Kidney,Embryonic/Fetal,Luciferase Reporter Assay,POSITIVE,DIRECT,DOWN,NaN,3p
109953,ENSG00000102978,POLR2C,hsa-mir-29c,Homo sapiens,MKN74,Stomach,Cancer/Malignant,Microarrays,NEGATIVE,INDIRECT,NaN,"24hrs post-transfection, Overexpression",3p
446282,ENSG00000167904,TMEM68,hsa-mir-95,Homo sapiens,MCF7,Mammary Gland,Cancer/Malignant,Microarrays,POSITIVE,INDIRECT,DOWN,"24hrs post-transfection, Overexpression",3p
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246354,ENSG00000130726,TRIM28,hsa-mir-30c-1,Homo sapiens,293S,Kidney,Embryonic/Fetal,HITS-CLIP,POSITIVE,DIRECT,DOWN,treatment:hippuristanol,3p
105785,ENSG00000101974,ATP11C,hsa-mir-3176,Homo sapiens,293S,Kidney,Embryonic/Fetal,HITS-CLIP,POSITIVE,DIRECT,DOWN,no treatment (control),NaN
496197,ENSG00000176903,PNMA1,hsa-mir-23a,Homo sapiens,BC1,Pleura,Cancer/Malignant,PAR-CLIP,POSITIVE,DIRECT,DOWN,NaN,3p
426081,ENSG00000165359,DDX26B,hsa-mir-182,Homo sapiens,Melanoma metastatic Liver Cells,Liver,Cancer/Malignant,Microarrays,NEGATIVE,INDIRECT,NaN,"Administered twice per week over 4 w, Silencing",5p


In [19]:
data.to_csv(edge_data_location + 'gene-miRNA_TarBase_v8_download.txt', header=None, sep='\t', index=None)

***
### miRNA-disease from [miR2Disease](http://watson.compbio.iupui.edu:8080/miR2Disease/)
miR2Disease, a manually curated database, aims at providing a comprehensive resource of miRNA deregulation in various human diseases.

In [20]:
data_downloader('http://watson.compbio.iupui.edu:8080/miR2Disease/download/AllEntries.txt', edge_data_location)
    
# Rename file adding relationship's identifier
os.rename(edge_data_location+'AllEntries.txt',
          edge_data_location+'miRNA-disease_miR2Disease.txt') 

with open(edge_data_location + 'miRNA-disease_mir2Disease.txt') as f:
    data = f.read()

data = pd.read_csv(edge_data_location + 'miRNA-disease_miR2Disease.txt', sep="\t", header=None)  
# For NCRO IDs compatability
data[0] = data[0].str.lower()
data

,0,1,2,3,4,5
0,hsa-let-7f-2,kidney cancer,up-regulated,microarray,2007.0,Micro-RNA profiling in kidney and bladder canc...
1,hsa-let-7g,hepatocellular carcinoma (HCC),down-regulated,"Northern blot, qRT-PCR etc",2008.0,Identification of metastasis-related microRNAs...
2,hsa-let-7g,lung cancer,down-regulated,"Northern blot, qRT-PCR etc",2007.0,The let-7 microRNA represses cell proliferatio...
3,hsa-let-7g,non-small cell lung cancer (NSCLC),down-regulated,"Northern blot, qRT-PCR etc",2008.0,Suppression of non-small cell lung tumor devel...
4,hsa-let-7g,ovarian cancer (OC),down-regulated,"Northern blot, qRT-PCR etc",2007.0,Let-7 expression defines two differentiation s...
...,...,...,...,...,...,...
2897,hsa-mir-21,glioblastoma multiforme (GBM),up-regulated,"Northern blot, qRT-PCR etc",2008.0,miR-124 and miR-137 inhibit proliferation of g...
2898,hsa-mir-21,glioma,up-regulated,"Northern blot, qRT-PCR etc",2008.0,MicroRNA 21 promotes glioma invasion by target...
2899,hsa-mir-21,hepatocellular carcinoma (HCC),up-regulated,"Northern blot, qRT-PCR etc",2006.0,Downregulation of miR-122 in the rodent and hu...
2900,hsa-mir-21,Inclusion body myositis (IBM),up-regulated,microarray,2007.0,Distinctive patterns of microRNA expression in...


In [21]:
# We need to move from description to DO Ontology IDs
data.columns = ['mirna', 'desc', 2,3,4,5]
disease2mirna = pd.merge(descDOmap, data, on=['desc'])
disease2mirna

,desc,doid,mirna,2,3,4,5
0,adenoma,DOID_657,hsa-let-7a,normal,"Northern blot, qRT-PCR etc",2007.0,Disrupting the pairing between let-7 and Hmga2...
1,adrenocortical carcinoma,DOID_3948,hsa-mir-184,up-regulated,microarray,2009.0,Integrative molecular-bioinformatics study of ...
2,adrenocortical carcinoma,DOID_3948,hsa-mir-503,up-regulated,microarray,2009.0,Integrative molecular-bioinformatics study of ...
3,adrenocortical carcinoma,DOID_3948,hsa-mir-511,down-regulated,microarray,2009.0,Integrative molecular-bioinformatics study of ...
4,adrenocortical carcinoma,DOID_3948,hsa-mir-214,down-regulated,microarray,2009.0,Integrative molecular-bioinformatics study of ...
...,...,...,...,...,...,...,...
1290,vascular disease,DOID_178,hsa-mir-21,up-regulated,"Northern blot, qRT-PCR etc",2007.0,MicroRNA expression signature and antisense-me...
1291,vascular disease,DOID_178,hsa-mir-352,up-regulated,"Northern blot, qRT-PCR etc",2007.0,MicroRNA expression signature and antisense-me...
1292,vascular disease,DOID_178,hsa-mir-365,down-regulated,"Northern blot, qRT-PCR etc",2007.0,MicroRNA expression signature and antisense-me...
1293,vesicular stomatitis,DOID_10881,hsa-mir-93,down-regulated,"Northern blot, qRT-PCR etc",2007.0,Hypersusceptibility to vesicular stomatitis vi...


In [22]:
disease2mirna.to_csv(edge_data_location + 'miRNA-disease_mir2Disease.txt', header=None, sep='\t', index=None)